In [82]:
import pandas as pd
from functools import reduce
from sqlalchemy import create_engine

In [83]:
# Read OECD education statistics csv file
csv_file = "Resources/OECD_education_stats.csv"
education_data_df = pd.read_csv(csv_file)
education_data_df.head()

GPS_CODE                         GPS variables LOCATION    Country  TIME  \
0     C002  Enrolment rate among 20-29 year-olds      AUS  Australia  1995   
1     C002  Enrolment rate among 20-29 year-olds      AUS  Australia  2000   
2     C002  Enrolment rate among 20-29 year-olds      AUS  Australia  2005   
3     C002  Enrolment rate among 20-29 year-olds      AUS  Australia  2006   
4     C002  Enrolment rate among 20-29 year-olds      AUS  Australia  2007   

   Time Unit Code        Unit  PowerCode Code PowerCode  \
0  1995        PC  Percentage               0     Units   
1  2000        PC  Percentage               0     Units   
2  2005        PC  Percentage               0     Units   
3  2006        PC  Percentage               0     Units   
4  2007        PC  Percentage               0     Units   

   Reference Period Code  Reference Period  Value Flag Codes Flags  
0                    NaN               NaN  23.13        NaN   NaN  
1                    NaN               NaN  28.23        NaN   NaN  
2                    NaN               NaN  33.20        NaN   NaN  
3                    NaN               NaN  33.18        NaN   NaN  
4                    NaN               NaN  33.07        NaN   NaN

In [84]:
# Select columns
oecd_ed_df = education_data_df[['GPS variables', 'LOCATION', 'Country','TIME','Value', 'Unit']]
oecd_ed_df.head()

GPS variables LOCATION    Country  TIME  Value  \
0  Enrolment rate among 20-29 year-olds      AUS  Australia  1995  23.13   
1  Enrolment rate among 20-29 year-olds      AUS  Australia  2000  28.23   
2  Enrolment rate among 20-29 year-olds      AUS  Australia  2005  33.20   
3  Enrolment rate among 20-29 year-olds      AUS  Australia  2006  33.18   
4  Enrolment rate among 20-29 year-olds      AUS  Australia  2007  33.07   

         Unit  
0  Percentage  
1  Percentage  
2  Percentage  
3  Percentage  
4  Percentage

In [85]:
# Check data types
oecd_ed_df.dtypes

# Check for NA values
oecd_ed_df.count() # Around 1200 values missing, but not dropping rows

GPS variables    4725
LOCATION         4725
Country          4725
TIME             4725
Value            3511
Unit             4725
dtype: int64

In [86]:
# Fix spelling errors (Enrolment >> Enrollment)
oecd_ed_df['GPS variables'] = oecd_ed_df['GPS variables'].replace(
    {'Enrolment rate among 20-29 year-olds':'Enrollment rate among 20-29 year-olds', 
     'Enrolment rate among 15-19 year-olds': 'Enrollment rate among 15-19 year-olds'
    })
oecd_ed_df.head()

/anaconda3/envs/PythonData2/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


GPS variables LOCATION    Country  TIME  Value  \
0  Enrollment rate among 20-29 year-olds      AUS  Australia  1995  23.13   
1  Enrollment rate among 20-29 year-olds      AUS  Australia  2000  28.23   
2  Enrollment rate among 20-29 year-olds      AUS  Australia  2005  33.20   
3  Enrollment rate among 20-29 year-olds      AUS  Australia  2006  33.18   
4  Enrollment rate among 20-29 year-olds      AUS  Australia  2007  33.07   

         Unit  
0  Percentage  
1  Percentage  
2  Percentage  
3  Percentage  
4  Percentage

In [87]:
# Rename columns
oecd_ed_df = oecd_ed_df.rename(
    columns={'GPS variables':'Variables',
             'LOCATION':'Country Code',
             'TIME':'Year',
              })

oecd_ed_df.head()

Variables Country Code    Country  Year  Value  \
0  Enrollment rate among 20-29 year-olds          AUS  Australia  1995  23.13   
1  Enrollment rate among 20-29 year-olds          AUS  Australia  2000  28.23   
2  Enrollment rate among 20-29 year-olds          AUS  Australia  2005  33.20   
3  Enrollment rate among 20-29 year-olds          AUS  Australia  2006  33.18   
4  Enrollment rate among 20-29 year-olds          AUS  Australia  2007  33.07   

         Unit  
0  Percentage  
1  Percentage  
2  Percentage  
3  Percentage  
4  Percentage

In [88]:
# Extract 2012 data
oecd_ed_2012=oecd_ed_df.loc[oecd_ed_df["Year"]==2012,:]

oecd_ed_2012.head()

Variables Country Code         Country  Year  \
9   Enrollment rate among 20-29 year-olds          AUS       Australia  2012   
19  Enrollment rate among 20-29 year-olds          AUT         Austria  2012   
29  Enrollment rate among 20-29 year-olds          BEL         Belgium  2012   
39  Enrollment rate among 20-29 year-olds          CAN          Canada  2012   
49  Enrollment rate among 20-29 year-olds          CZE  Czech Republic  2012   

    Value        Unit  
9   34.52  Percentage  
19  25.85  Percentage  
29  32.62  Percentage  
39    NaN  Percentage  
49  25.98  Percentage

In [89]:
oecd_ed_2012=oecd_ed_2012.reset_index(drop=True)
oecd_ed_2012.head()

Variables Country Code         Country  Year  \
0  Enrollment rate among 20-29 year-olds          AUS       Australia  2012   
1  Enrollment rate among 20-29 year-olds          AUT         Austria  2012   
2  Enrollment rate among 20-29 year-olds          BEL         Belgium  2012   
3  Enrollment rate among 20-29 year-olds          CAN          Canada  2012   
4  Enrollment rate among 20-29 year-olds          CZE  Czech Republic  2012   

   Value        Unit  
0  34.52  Percentage  
1  25.85  Percentage  
2  32.62  Percentage  
3    NaN  Percentage  
4  25.98  Percentage

In [90]:
# Create a dataframe for Enrollment rate among 20-29 year-olds
oecd_enroll_20=oecd_ed_2012.loc[oecd_ed_2012["Variables"]=="Enrollment rate among 20-29 year-olds",:]

# Change value column name to variable name and drop variable column
oecd_enroll_20=oecd_enroll_20.rename(columns={"Value": "Enrollment Rate Age 20-29"})
oecd_enroll_20=oecd_enroll_20.drop(['Variables', 'Unit'], axis=1)

oecd_enroll_20.head()

Country Code         Country  Year  Enrollment Rate Age 20-29
0          AUS       Australia  2012                      34.52
1          AUT         Austria  2012                      25.85
2          BEL         Belgium  2012                      32.62
3          CAN          Canada  2012                        NaN
4          CZE  Czech Republic  2012                      25.98

In [91]:
# Create dataframe for enrollment rate among 15-19 year-olds
oecd_enroll_15=oecd_ed_2012.loc[oecd_ed_2012["Variables"]=="Enrollment rate among 15-19 year-olds",:]

# Change value column name to variable name and drop variable and unit columns
oecd_enroll_15=oecd_enroll_15.rename(columns={"Value": "Enrollment Rate Age 15-19"})
oecd_enroll_15=oecd_enroll_15.drop(['Variables', 'Unit', 'Country', 'Year'], axis=1)

oecd_enroll_15.head()

Country Code  Enrollment Rate Age 15-19
45          AUS                      86.52
46          AUT                      78.85
47          BEL                      93.96
48          CAN                        NaN
49          CZE                      90.17

In [92]:
# Create dataframe for expected years of education for women
oecd_women_years=oecd_ed_2012.loc[oecd_ed_2012["Variables"]=="Expected number of years in education for women (All levels combined, Full-time), from age 5 through age 39",:]

# Change value column name to variable name and drop variable and unit columns
oecd_women_years=oecd_women_years.rename(columns={"Value": "Expected Education For Women (Years)"})
oecd_women_years=oecd_women_years.drop(['Variables', 'Unit', 'Country', 'Year'], axis=1)

oecd_women_years.head()

Country Code  Expected Education For Women (Years)
585          AUS                                 16.21
586          AUT                                 17.28
587          BEL                                 16.57
588          CAN                                 15.85
589          CZE                                 17.94

In [93]:
# Create dataframe for expected years of education for women
oecd_men_years=oecd_ed_2012.loc[oecd_ed_2012["Variables"]=="Expected number of years in education for men (All levels combined, Full-time), from age 5 through age 39",:]

# Change value column name to variable name and drop variable and unit columns
oecd_men_years=oecd_men_years.rename(columns={"Value": "Expected Education For Men (Years)"})
oecd_men_years=oecd_men_years.drop(['Variables', 'Unit', 'Country', 'Year'], axis=1)

oecd_men_years.head()

Country Code  Expected Education For Men (Years)
540          AUS                               15.52
541          AUT                               16.75
542          BEL                               16.08
543          CAN                               15.15
544          CZE                               17.12

In [94]:
# Merge the four formatted dataframes together, set index
oecd_ed_stats_2012 = oecd_enroll_20.merge(oecd_enroll_15, on='Country Code').merge(oecd_women_years, on='Country Code').merge(oecd_men_years, on='Country Code')
oecd_ed_stats_2012.set_index("Country Code", inplace=True)

oecd_ed_stats_2012.head()

Country  Year  Enrollment Rate Age 20-29  \
Country Code                                                    
AUS                Australia  2012                      34.52   
AUT                  Austria  2012                      25.85   
BEL                  Belgium  2012                      32.62   
CAN                   Canada  2012                        NaN   
CZE           Czech Republic  2012                      25.98   

              Enrollment Rate Age 15-19  Expected Education For Women (Years)  \
Country Code                                                                    
AUS                               86.52                                 16.21   
AUT                               78.85                                 17.28   
BEL                               93.96                                 16.57   
CAN                                 NaN                                 15.85   
CZE                               90.17                                 17.94   

              Expected Education For Men (Years)  
Country Code                                      
AUS                                        15.52  
AUT                                        16.75  
BEL                                        16.08  
CAN                                        15.15  
CZE                                        17.12

In [95]:
# Connect to local database
#database_path = "etl"
#engine = create_engine(f"sqlite:///{database_path}")

In [96]:
# Check for tables
#engine.table_names()

In [97]:
# Load converted OECD CSV table into database
#oecd_ed_stats_2012.to_sql(name='oecd_education_data', con=engine, if_exists='replace', index=True)

In [98]:
# Confirm data has been loaded
#pd.read_sql_query('select * from oecd_education_data', con=engine).head(10)

In [99]:
# Check for table names
#engine.table_names()

In [100]:
# Create a reference the CSV file desired
csv_path = "Resources/WorldBankData.csv"

# Read the CSV into a Pandas DataFrame
wbd_df = pd.read_csv(csv_path)

# Print the first five rows of data to the screen
wbd_df.head(60)

Country Name Country Code  \
0     Australia          AUS   
1     Australia          AUS   
2     Australia          AUS   
3     Australia          AUS   
4     Australia          AUS   
5     Australia          AUS   
6     Australia          AUS   
7     Australia          AUS   
8     Australia          AUS   
9     Australia          AUS   
10    Australia          AUS   
11    Australia          AUS   
12    Australia          AUS   
13    Australia          AUS   
14    Australia          AUS   
15    Australia          AUS   
16    Australia          AUS   
17    Australia          AUS   
18    Australia          AUS   
19    Australia          AUS   
20    Australia          AUS   
21    Australia          AUS   
22    Australia          AUS   
23    Australia          AUS   
24    Australia          AUS   
25    Australia          AUS   
26    Australia          AUS   
27    Australia          AUS   
28    Australia          AUS   
29    Australia          AUS   
30    Australia          AUS   
31    Australia          AUS   
32    Australia          AUS   
33    Australia          AUS   
34    Australia          AUS   
35    Australia          AUS   
36    Australia          AUS   
37    Australia          AUS   
38    Australia          AUS   
39    Australia          AUS   
40    Australia          AUS   
41    Australia          AUS   
42    Australia          AUS   
43    Australia          AUS   
44    Australia          AUS   
45    Australia          AUS   
46    Australia          AUS   
47    Australia          AUS   
48    Australia          AUS   
49    Australia          AUS   
50    Australia          AUS   
51    Australia          AUS   
52    Australia          AUS   
53    Australia          AUS   
54    Australia          AUS   
55       France          FRA   
56       France          FRA   
57       France          FRA   
58       France          FRA   
59       France          FRA   

                                          Series Name        Series Code  \
0   Adolescent fertility rate (births per 1,000 wo...        SP.ADO.TFRT   
1   Agriculture, forestry, and fishing, value adde...     NV.AGR.TOTL.ZS   
2   Annual freshwater withdrawals, total (% of int...     ER.H2O.FWTL.ZS   
3   Births attended by skilled health staff (% of ...     SH.STA.BRTC.ZS   
4              CO2 emissions (metric tons per capita)     EN.ATM.CO2E.PC   
5   Contraceptive prevalence, any methods (% of wo...     SP.DYN.CONU.ZS   
6   Domestic credit provided by financial sector (...  FS.AST.DOMS.GD.ZS   
7         Electric power consumption (kWh per capita)  EG.USE.ELEC.KH.PC   
8        Energy use (kg of oil equivalent per capita)  EG.USE.PCAP.KG.OE   
9            Exports of goods and services (% of GDP)     NE.EXP.GNFS.ZS   
10     External debt stocks, total (DOD, current US$)     DT.DOD.DECT.CD   
11           Fertility rate, total (births per woman)     SP.DYN.TFRT.IN   
12  Foreign direct investment, net inflows (BoP, c...  BX.KLT.DINV.CD.WD   
13                               Forest area (sq. km)     AG.LND.FRST.K2   
14                                  GDP (current US$)     NY.GDP.MKTP.CD   
15                              GDP growth (annual %)  NY.GDP.MKTP.KD.ZG   
16         GNI per capita, Atlas method (current US$)     NY.GNP.PCAP.CD   
17      GNI per capita, PPP (current international $)  NY.GNP.PCAP.PP.CD   
18                    GNI, Atlas method (current US$)     NY.GNP.ATLS.CD   
19                 GNI, PPP (current international $)  NY.GNP.MKTP.PP.CD   
20                 Gross capital formation (% of GDP)     NE.GDI.TOTL.ZS   
21  High-technology exports (% of manufactured exp...  TX.VAL.TECH.MF.ZS   
22  Immunization, measles (% of children ages 12-2...        SH.IMM.MEAS   
23           Imports of goods and services (% of GDP)     NE.IMP.GNFS.ZS   
24                    Income share held by lowest 20%     SI.DST.FRST.20   
25  Industry (including construction), value added...     NV.IND.TOTL.ZS  

In [101]:
#del wbd_df ["Series Code"]
del wbd_df ["2009 [YR2009]"]
del wbd_df ["2010 [YR2010]"]
del wbd_df ["2011 [YR2011]"]
del wbd_df ["2013 [YR2013]"]
del wbd_df ["2014 [YR2014]"]
del wbd_df ["2015 [YR2015]"]
del wbd_df ["2016 [YR2016]"]
del wbd_df ["2017 [YR2017]"]
del wbd_df ["2018 [YR2018]"]

wbd_df.head()

Country Name Country Code  \
0    Australia          AUS   
1    Australia          AUS   
2    Australia          AUS   
3    Australia          AUS   
4    Australia          AUS   

                                         Series Name     Series Code  \
0  Adolescent fertility rate (births per 1,000 wo...     SP.ADO.TFRT   
1  Agriculture, forestry, and fishing, value adde...  NV.AGR.TOTL.ZS   
2  Annual freshwater withdrawals, total (% of int...  ER.H2O.FWTL.ZS   
3  Births attended by skilled health staff (% of ...  SH.STA.BRTC.ZS   
4             CO2 emissions (metric tons per capita)  EN.ATM.CO2E.PC   

  2012 [YR2012]  
0        14.801  
1   2.258752004  
2   3.140243902  
3            ..  
4   17.07290468

In [102]:
wbd_df = wbd_df.rename(columns={'Country Name': 'Country_Name', 'Country Code': 'Country_Code', 'Series Name': 'Series_Name', 'Series Code': 'Series_Code'})



wbd_df.head()

Country_Name Country_Code  \
0    Australia          AUS   
1    Australia          AUS   
2    Australia          AUS   
3    Australia          AUS   
4    Australia          AUS   

                                         Series_Name     Series_Code  \
0  Adolescent fertility rate (births per 1,000 wo...     SP.ADO.TFRT   
1  Agriculture, forestry, and fishing, value adde...  NV.AGR.TOTL.ZS   
2  Annual freshwater withdrawals, total (% of int...  ER.H2O.FWTL.ZS   
3  Births attended by skilled health staff (% of ...  SH.STA.BRTC.ZS   
4             CO2 emissions (metric tons per capita)  EN.ATM.CO2E.PC   

  2012 [YR2012]  
0        14.801  
1   2.258752004  
2   3.140243902  
3            ..  
4   17.07290468

In [103]:
#wbd_3_df = wbd_df.loc[wbd_df["Series_Code"]=="SH.STA.BRTC.ZS",:]
wbd_6_df = wbd_df.loc[wbd_df["Series_Code"]=="FS.AST.DOMS.GD.ZS",:]
#wbd_10_df = wbd_df.loc[wbd_df["Series_Code"]=="DT.DOD.DECT.CD",:]
wbd_11_df = wbd_df.loc[wbd_df["Series_Code"]=="SP.DYN.TFRT.IN",:]
wbd_16_df = wbd_df.loc[wbd_df["Series_Code"]=="NY.GNP.PCAP.CD",:]
wbd_17_df = wbd_df.loc[wbd_df["Series_Code"]=="NY.GNP.PCAP.PP.CD",:]
wbd_31_df = wbd_df.loc[wbd_df["Series_Code"]=="SH.DYN.MORT",:]
#wbd_40_df = wbd_df.loc[wbd_df["Series_Code"]=="SI.POV.NAHC",:]
wbd_52_df = wbd_df.loc[wbd_df["Series_Code"]=="IC.REG.DURS",:]

In [104]:
wbd_merge_1 = [wbd_6_df, wbd_11_df]

In [105]:
wbd_merge_1 = reduce(lambda left,right: pd.merge(left,right,on=["Country_Name","Country_Code"]), wbd_merge_1)

wbd_merge_1.head()

Country_Name Country_Code  \
0    Australia          AUS   
1       France          FRA   
2      Germany          DEU   
3        Italy          ITA   
4        Japan          JPN   

                                       Series_Name_x      Series_Code_x  \
0  Domestic credit provided by financial sector (...  FS.AST.DOMS.GD.ZS   
1  Domestic credit provided by financial sector (...  FS.AST.DOMS.GD.ZS   
2  Domestic credit provided by financial sector (...  FS.AST.DOMS.GD.ZS   
3  Domestic credit provided by financial sector (...  FS.AST.DOMS.GD.ZS   
4  Domestic credit provided by financial sector (...  FS.AST.DOMS.GD.ZS   

  2012 [YR2012]_x                             Series_Name_y   Series_Code_y  \
0     150.1579391  Fertility rate, total (births per woman)  SP.DYN.TFRT.IN   
1     147.6736927  Fertility rate, total (births per woman)  SP.DYN.TFRT.IN   
2     156.9249454  Fertility rate, total (births per woman)  SP.DYN.TFRT.IN   
3     177.3916251  Fertility rate, total (births per woman)  SP.DYN.TFRT.IN   
4     329.7278431  Fertility rate, total (births per woman)  SP.DYN.TFRT.IN   

  2012 [YR2012]_y  
0            1.92  
1            2.01  
2            1.41  
3            1.43  
4            1.41

In [106]:
del wbd_merge_1 ["Series_Code_y"]
del wbd_merge_1 ["Series_Code_x"]
del wbd_merge_1 ["Series_Name_y"]
del wbd_merge_1 ["Series_Name_x"]

In [107]:
wbd_merge_1 = wbd_merge_1.rename(columns={"2012 [YR2012]_x": 'Domestic Credit', "2012 [YR2012]_y": "Fertility Rate"})
wbd_merge_1.head()

Country_Name Country_Code Domestic Credit Fertility Rate
0    Australia          AUS     150.1579391           1.92
1       France          FRA     147.6736927           2.01
2      Germany          DEU     156.9249454           1.41
3        Italy          ITA     177.3916251           1.43
4        Japan          JPN     329.7278431           1.41

In [108]:
wbd_merge_2 = [wbd_16_df, wbd_17_df]

In [109]:
wbd_merge_2 = reduce(lambda left,right: pd.merge(left,right,on=["Country_Name","Country_Code"]), wbd_merge_2)

wbd_merge_2.head()

Country_Name Country_Code                               Series_Name_x  \
0    Australia          AUS  GNI per capita, Atlas method (current US$)   
1       France          FRA  GNI per capita, Atlas method (current US$)   
2      Germany          DEU  GNI per capita, Atlas method (current US$)   
3        Italy          ITA  GNI per capita, Atlas method (current US$)   
4        Japan          JPN  GNI per capita, Atlas method (current US$)   

    Series_Code_x 2012 [YR2012]_x  \
0  NY.GNP.PCAP.CD           60090   
1  NY.GNP.PCAP.CD           43380   
2  NY.GNP.PCAP.CD           46710   
3  NY.GNP.PCAP.CD           36000   
4  NY.GNP.PCAP.CD           49480   

                                   Series_Name_y      Series_Code_y  \
0  GNI per capita, PPP (current international $)  NY.GNP.PCAP.PP.CD   
1  GNI per capita, PPP (current international $)  NY.GNP.PCAP.PP.CD   
2  GNI per capita, PPP (current international $)  NY.GNP.PCAP.PP.CD   
3  GNI per capita, PPP (current international $)  NY.GNP.PCAP.PP.CD   
4  GNI per capita, PPP (current international $)  NY.GNP.PCAP.PP.CD   

  2012 [YR2012]_y  
0           41450  
1           38500  
2           44590  
3           36190  
4           38190

In [110]:
wbd_merge_2 = wbd_merge_2.rename(columns={"2012 [YR2012]_x": 'GNI Per Capita', "2012 [YR2012]_y": "GNI Per Capita, PPP"})
wbd_merge_2.head()

Country_Name Country_Code                               Series_Name_x  \
0    Australia          AUS  GNI per capita, Atlas method (current US$)   
1       France          FRA  GNI per capita, Atlas method (current US$)   
2      Germany          DEU  GNI per capita, Atlas method (current US$)   
3        Italy          ITA  GNI per capita, Atlas method (current US$)   
4        Japan          JPN  GNI per capita, Atlas method (current US$)   

    Series_Code_x GNI Per Capita  \
0  NY.GNP.PCAP.CD          60090   
1  NY.GNP.PCAP.CD          43380   
2  NY.GNP.PCAP.CD          46710   
3  NY.GNP.PCAP.CD          36000   
4  NY.GNP.PCAP.CD          49480   

                                   Series_Name_y      Series_Code_y  \
0  GNI per capita, PPP (current international $)  NY.GNP.PCAP.PP.CD   
1  GNI per capita, PPP (current international $)  NY.GNP.PCAP.PP.CD   
2  GNI per capita, PPP (current international $)  NY.GNP.PCAP.PP.CD   
3  GNI per capita, PPP (current international $)  NY.GNP.PCAP.PP.CD   
4  GNI per capita, PPP (current international $)  NY.GNP.PCAP.PP.CD   

  GNI Per Capita, PPP  
0               41450  
1               38500  
2               44590  
3               36190  
4               38190

In [111]:
del wbd_merge_2 ["Series_Code_y"]
del wbd_merge_2 ["Series_Code_x"]
del wbd_merge_2 ["Series_Name_y"]
del wbd_merge_2 ["Series_Name_x"]
wbd_merge_2.head()

Country_Name Country_Code GNI Per Capita GNI Per Capita, PPP
0    Australia          AUS          60090               41450
1       France          FRA          43380               38500
2      Germany          DEU          46710               44590
3        Italy          ITA          36000               36190
4        Japan          JPN          49480               38190

In [112]:
wbd_merge_3 = [wbd_31_df, wbd_52_df]

In [113]:
wbd_merge_3 = reduce(lambda left,right: pd.merge(left,right,on=["Country_Name","Country_Code"]), wbd_merge_3)
wbd_merge_3.head()

Country_Name Country_Code                                    Series_Name_x  \
0    Australia          AUS  Mortality rate, under-5 (per 1,000 live births)   
1       France          FRA  Mortality rate, under-5 (per 1,000 live births)   
2      Germany          DEU  Mortality rate, under-5 (per 1,000 live births)   
3        Italy          ITA  Mortality rate, under-5 (per 1,000 live births)   
4        Japan          JPN  Mortality rate, under-5 (per 1,000 live births)   

  Series_Code_x 2012 [YR2012]_x                             Series_Name_y  \
0   SH.DYN.MORT             4.3  Time required to start a business (days)   
1   SH.DYN.MORT             4.1  Time required to start a business (days)   
2   SH.DYN.MORT               4  Time required to start a business (days)   
3   SH.DYN.MORT             3.8  Time required to start a business (days)   
4   SH.DYN.MORT               3  Time required to start a business (days)   

  Series_Code_y 2012 [YR2012]_y  
0   IC.REG.DURS             2.5  
1   IC.REG.DURS             6.5  
2   IC.REG.DURS            14.5  
3   IC.REG.DURS             7.5  
4   IC.REG.DURS              ..

In [114]:
wbd_merge_3 = wbd_merge_3.rename(columns={"2012 [YR2012]_x": 'Mortality Rate Under 5', "2012 [YR2012]_y": "Days to Start a Business"})
wbd_merge_3.head()

Country_Name Country_Code                                    Series_Name_x  \
0    Australia          AUS  Mortality rate, under-5 (per 1,000 live births)   
1       France          FRA  Mortality rate, under-5 (per 1,000 live births)   
2      Germany          DEU  Mortality rate, under-5 (per 1,000 live births)   
3        Italy          ITA  Mortality rate, under-5 (per 1,000 live births)   
4        Japan          JPN  Mortality rate, under-5 (per 1,000 live births)   

  Series_Code_x Mortality Rate Under 5  \
0   SH.DYN.MORT                    4.3   
1   SH.DYN.MORT                    4.1   
2   SH.DYN.MORT                      4   
3   SH.DYN.MORT                    3.8   
4   SH.DYN.MORT                      3   

                              Series_Name_y Series_Code_y  \
0  Time required to start a business (days)   IC.REG.DURS   
1  Time required to start a business (days)   IC.REG.DURS   
2  Time required to start a business (days)   IC.REG.DURS   
3  Time required to start a business (days)   IC.REG.DURS   
4  Time required to start a business (days)   IC.REG.DURS   

  Days to Start a Business  
0                      2.5  
1                      6.5  
2                     14.5  
3                      7.5  
4                       ..

In [115]:
del wbd_merge_3 ["Series_Code_y"]
del wbd_merge_3 ["Series_Code_x"]
del wbd_merge_3 ["Series_Name_y"]
del wbd_merge_3 ["Series_Name_x"]
wbd_merge_3.head()

Country_Name Country_Code Mortality Rate Under 5 Days to Start a Business
0    Australia          AUS                    4.3                      2.5
1       France          FRA                    4.1                      6.5
2      Germany          DEU                      4                     14.5
3        Italy          ITA                    3.8                      7.5
4        Japan          JPN                      3                       ..

In [116]:
wbd_merge_all = [wbd_merge_1, wbd_merge_2, wbd_merge_3]

In [117]:
wbd_merge_all = reduce(lambda left,right: pd.merge(left,right,on=["Country_Name","Country_Code"]), wbd_merge_all)
wbd_merge_all.head(40)

Country_Name Country_Code Domestic Credit Fertility Rate  \
0         Australia          AUS     150.1579391           1.92   
1            France          FRA     147.6736927           2.01   
2           Germany          DEU     156.9249454           1.41   
3             Italy          ITA     177.3916251           1.43   
4             Japan          JPN     329.7278431           1.41   
5       Korea, Rep.          KOR     155.8450252          1.297   
6            Mexico          MEX     45.63480336          2.295   
7       Netherlands          NLD     231.3754468           1.72   
8             Spain          ESP     245.5703178           1.32   
9       Switzerland          CHE     182.2955505           1.52   
10           Turkey          TUR     67.14388959          2.124   
11   United Kingdom          GBR     191.1396689           1.92   
12    United States          USA     232.9158893         1.8805   
13     OECD members          OED     206.0030021     1.76708779   
14          Austria          AUT     135.2536916           1.44   
15          Belgium          BEL     113.2345514            1.8   
16           Canada          CAN              ..         1.6142   
17            Chile          CHL     112.5324815          1.834   
18   Czech Republic          CZE     68.36776265           1.45   
19          Denmark          DNK     230.4667752           1.73   
20          Estonia          EST     75.31687269           1.56   
21          Finland          FIN     165.4192089            1.8   
22           Greece          GRC     139.1190185           1.34   
23          Hungary          HUN     68.18051376           1.34   
24          Iceland          ISL     150.7532492           2.04   
25          Ireland          IRL     205.1437156           1.98   
26           Israel          ISR     85.90979646           3.05   
27           Latvia          LVA     63.89585706           1.44   
28        Lithuania          LTU     51.94840057            1.6   
29       Luxembourg          LUX     204.6694903           1.57   
30      New Zealand          NZL              ..            2.1   
31           Norway          NOR     123.2378048           1.85   
32           Poland          POL     64.06689476           1.33   
33         Portugal          PRT     201.2482704           1.28   
34  Slovak Republic          SVK     62.96193693           1.34   
35         Slovenia          SVN     92.58259469           1.58   
36           Sweden          SWE     156.3847584           1.91   

   GNI Per Capita GNI Per Capita, PPP Mortality Rate Under 5  \
0           60090               41450                    4.3   
1           43380               38500                    4.1   
2           46710               44590                      4   
3           36000               36190                    3.8   
4           49480               38190                      3   
5           24550               32430                    3.8   
6            9750               16340                   16.4   
7           52930               48080                    4.2   
8           29760               31760                    3.6   
9           84880               59140                    4.4   
10          11880               20480                   16.6   
11          41310               37510                    4.8   
12          52760               53110                    7.1   
13    38679.27969          37959.2093            7.893377789   
14          50060               46500                    4.1   
15          47280               43470                    4.3   
16          50900               41400                    5.5   
17          14420               20600                    8.4   
18          19350               27250                    3.2   
19          61650               45820                    4.1   
20          16900               24970                    3.9   
21          48670               40830               

In [118]:
# Connect to local database
database_path = "etl"
engine = create_engine(f"sqlite:///{database_path}")

In [119]:
# Check for tables
engine.table_names()

['oecd_education_data']

In [120]:
# Load converted OECD CSV table into database
oecd_ed_stats_2012.to_sql(name='oecd_education_data', con=engine, if_exists='replace', index=True)

In [121]:
# Confirm data has been loaded
pd.read_sql_query('select * from oecd_education_data', con=engine).head(10)

Country Code         Country  Year  Enrollment Rate Age 20-29  \
0          AUS       Australia  2012                      34.52   
1          AUT         Austria  2012                      25.85   
2          BEL         Belgium  2012                      32.62   
3          CAN          Canada  2012                        NaN   
4          CZE  Czech Republic  2012                      25.98   
5          DNK         Denmark  2012                      43.25   
6          FIN         Finland  2012                      41.76   
7          FRA          France  2012                      20.93   
8          DEU         Germany  2012                      33.07   
9          GRC          Greece  2012                      41.50   

   Enrollment Rate Age 15-19  Expected Education For Women (Years)  \
0                      86.52                                 16.21   
1                      78.85                                 17.28   
2                      93.96                                 16.57   
3                        NaN                                 15.85   
4                      90.17                                 17.94   
5                      87.38                                 19.25   
6                      85.92                                 18.54   
7                      83.63                                 16.77   
8                      89.68                                 17.62   
9                      85.33                                 18.54   

   Expected Education For Men (Years)  
0                               15.52  
1                               16.75  
2                               16.08  
3                               15.15  
4                               17.12  
5                               18.50  
6                               17.54  
7                               16.13  
8                               17.89  
9                               18.21

In [122]:
# Check for table names
engine.table_names()

['oecd_education_data']